# timeit

In [75]:
import timeit

In [76]:
%timeit subset_1 = my_stat[(my_stat['V1'] > 0) & (my_stat['V3'] == 'A')]
%timeit subset_1 = my_stat.query("(V1 > 0) and (V3 == 'A')" )
# 883 µs ± 4.69 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
# 1.87 ms ± 4.03 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

%timeit subset_2 = my_stat[(my_stat['V2'] != 10) | (my_stat['V4'] >= 1)]
%timeit subset_2 = my_stat.query("(V1 > 0) and (V3 == 'A')" )
# 865 µs ± 18.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
# 1.91 ms ± 34.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)

NameError: name 'my_stat' is not defined

# timer

In [36]:
import sys
import time
reps = 1000
repslist = range(reps)
def timer(func, *pargs, **kargs):
    start = time.process_time()
    for i in repslist:
        ret = func(*pargs, **kargs)
    elapsed = time.process_time() - start
    return (elapsed, ret)

In [37]:
def forLoop():
    res = []
    for x in repslist:
        res.append(abs(x))
    return res
def listComp():
    return [abs(x) for x in repslist]
def mapCall():
    return list(map(abs, repslist)) # Вызов list необходим только в 3.0
def genExpr():
    return list(abs(x) for x in repslist) # Функция list вынуждает
reps = 100
repslist = range(reps)
def genFunc():
    def gen():
        for x in repslist:
            yield abs(x)
    return list(gen())

In [38]:
print(sys.version)
for test in (forLoop, listComp, mapCall, genExpr, genFunc):
    elapsed, result = timer(test)
    print ('-' * 33)
    print ('%-9s: %.5f => [%s...%s]' %
            (test.__name__, elapsed, result[0], result[-1]))

3.7.3 (default, Apr 24 2019, 15:29:51) [MSC v.1915 64 bit (AMD64)]
---------------------------------
forLoop  : 0.09360 => [0...999]
---------------------------------
listComp : 0.06240 => [0...999]
---------------------------------
mapCall  : 0.03120 => [0...999]
---------------------------------
genExpr  : 0.09360 => [0...999]
---------------------------------
genFunc  : 0.07800 => [0...999]


In [39]:
def forLoop():
    res = []
    for x in repslist:
        res.append(x + 10)
    return res
def listComp():
    return [x + 10 for x in repslist]
def mapCall():
    return list(map((lambda x: x + 10), repslist)) # Вызов list необходим
# только в 3.0
def genExpr():
    return list(x + 10 for x in repslist) # Вызов list необходим в 2.6 + 3.0
def genFunc():
    def gen():
        for x in repslist:
            yield x + 10
    return list(gen())

In [40]:
print(sys.version)
for test in (forLoop, listComp, mapCall, genExpr, genFunc):
    elapsed, result = timer(test)
    print ('-' * 33)
    print ('%-9s: %.5f => [%s...%s]' %
            (test.__name__, elapsed, result[0], result[-1]))

3.7.3 (default, Apr 24 2019, 15:29:51) [MSC v.1915 64 bit (AMD64)]
---------------------------------
forLoop  : 0.09360 => [10...1009]
---------------------------------
listComp : 0.04680 => [10...1009]
---------------------------------
mapCall  : 0.09360 => [10...1009]
---------------------------------
genExpr  : 0.07800 => [10...1009]
---------------------------------
genFunc  : 0.06240 => [10...1009]


In [60]:
import time, sys
if sys.platform[:3] == 'win':
    timefunc = time.process_time # В Windows использовать
else:
    timefunc = time.process_time # На некоторых платформах Unix дает
# лучшее разрешение
def trace(*args): pass # Заглушка: вывод аргументов
def timer(func, *pargs, **kargs):
    _reps = kargs.pop('_reps', 1000) # Полученное число повторов
# или значение по умолчанию
    trace(func, pargs, kargs, _reps)
    repslist = range(_reps) # Вызов range вынесен за пределы
# цикла for для версии 2.6
    start = timefunc()
    for i in repslist:
        ret = func(*pargs, **kargs)
    elapsed = timefunc() - start
    return (elapsed, ret)

def best(func, *pargs, **kargs):
    _reps = kargs.pop('_reps', 50)
    best = 2 ** 32
    for i in range(_reps):
        (time, ret) = timer(func, *pargs, _reps=1, **kargs)
        if time < best: best = time
    return (best, ret)

In [59]:
import sys
reps = 10000
repslist = range(reps)
print(sys.version)
for tester in (timer, best):
    print('<%s>' % tester.__name__)
    for test in (forLoop, listComp, mapCall, genExpr, genFunc):
        elapsed, result = tester(test)
        print ('-' * 35)
        print ('%-9s: %.5f => [%s...%s]' %
            (test.__name__, elapsed, result[0], result[-1]))

3.7.3 (default, Apr 24 2019, 15:29:51) [MSC v.1915 64 bit (AMD64)]
<timer>
-----------------------------------
forLoop  : 0.84241 => [10...10009]
-----------------------------------
listComp : 0.53040 => [10...10009]
-----------------------------------
mapCall  : 0.99841 => [10...10009]
-----------------------------------
genExpr  : 0.73320 => [10...10009]
-----------------------------------
genFunc  : 0.73320 => [10...10009]
<best>
-----------------------------------
forLoop  : 0.00000 => [10...10009]
-----------------------------------
listComp : 0.00000 => [10...10009]
-----------------------------------
mapCall  : 0.00000 => [10...10009]
-----------------------------------
genExpr  : 0.00000 => [10...10009]
-----------------------------------
genFunc  : 0.00000 => [10...10009]


In [61]:
>>> def power(X, Y): return X ** Y # Испытуемая функция
...
>>> timer(power, 2, 32) # Общее время, последний результат

(0.0, 4294967296)

In [62]:
>>> timer(power, 2, 32, _reps=1000000)

(0.3588022999999936, 4294967296)

In [65]:
>>> timer(power, 2, 100000)[0]

0.24960160000000542

In [66]:
>>> best(power, 2, 32)

(0.0, 4294967296)

In [67]:
>>> best(power, 2, 100000)[0]

0.0

In [68]:
>>> best(power, 2, 100000, _reps=500)[0]

0.0

In [69]:
>>> timer(power, 2, 1000000, _reps=1)[0]

0.015600100000000339

In [70]:
>>> timer(power, 2, 1000000, _reps=10)[0]

0.031200200000000677

In [74]:
>>> best(power, 2, 1000000, _reps=1)[0]

0.0

In [72]:
>>> best(power, 2, 1000000, _reps=10)[0]

0.0

In [73]:
>>> best(power, 2, 1000000, _reps=50)[0] 

0.0